In [1]:
# import some Python dependencies

import urllib2
import json
import datetime
import csv
import time

In [8]:

# Since the code output in this notebook leaks the app_secret,
# it has been reset by the time you read this.

app_id = "get your own code https://developers.facebook.com/docs/javascript/reference/FB.api/
app_secret = "" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

In [9]:
page_id = 'Mochablogger'

In [10]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    print(url)
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageData(page_id, access_token)

https://graph.facebook.com/v2.4/Mochablogger/?access_token=931781180288750|2fd24e03ee59309df867b998f225d346&fb
{
    "id": "319779186521", 
    "name": "MOCHA USON BLOG"
}


In [12]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

In [13]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageFeedData(page_id, access_token)

{
    "data": [
        {
            "created_time": "2016-10-16T07:35:18+0000", 
            "id": "319779186521_10154678860731522", 
            "message": "Eh di lumabas din tunay na ugali ni MOCHA USON BLOG! Siya daw may-ari ng Pilipinas kaya nagpapalayas na dito kapag ayae kay Demonyong DuTae! Kapag talaga ng mukha ng tuwadera na yan, balang-araw may paglalagayan ka din wahahhahaha"
        }, 
        {
            "created_time": "2016-10-16T07:31:26+0000", 
            "id": "319779186521_10155328355743294", 
            "message": "Ka-DDS bakit nga ba super duper sikat popular famous to the max ang ating mahal na pangulong Duterte sa buong mundo? Anong meron siya na wala sa ibang presidente sa ibang bansa? Please read, watch, share this post and video and be INSPIRED!"
        }, 
        {
            "created_time": "2016-10-16T07:30:47+0000", 
            "id": "319779186521_1347001212001375", 
            "message": "Permission to post Po miss mocha, this is what's happen

In [14]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print json.dumps(test_status, indent=4, sort_keys=True)

{
    "comments": {
        "data": [], 
        "summary": {
            "can_comment": false, 
            "order": "chronological", 
            "total_count": 0
        }
    }, 
    "created_time": "2016-10-16T07:35:18+0000", 
    "id": "319779186521_10154678860731522", 
    "likes": {
        "data": [], 
        "summary": {
            "can_like": false, 
            "has_liked": false, 
            "total_count": 0
        }
    }, 
    "message": "Eh di lumabas din tunay na ugali ni MOCHA USON BLOG! Siya daw may-ari ng Pilipinas kaya nagpapalayas na dito kapag ayae kay Demonyong DuTae! Kapag talaga ng mukha ng tuwadera na yan, balang-araw may paglalagayan ka din wahahhahaha", 
    "type": "status"
}


In [15]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print processed_test_status

(u'319779186521_10154678860731522', 'Eh di lumabas din tunay na ugali ni MOCHA USON BLOG! Siya daw may-ari ng Pilipinas kaya nagpapalayas na dito kapag ayae kay Demonyong DuTae! Kapag talaga ng mukha ng tuwadera na yan, balang-araw may paglalagayan ka din wahahhahaha', '', u'status', '', '2016-10-16 02:35:18', 0, 0, 0)


In [16]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_fb_posts.csv' % page_id, 'wb', encoding='utf-8') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                encode('ascii', 'ignore')
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)




In [ ]:
scrapeFacebookPageFeedStatus(page_id, access_token)